<font size="6">Улучшение сходимости нейросетей и борьба с переобучением</font>

# Сигмоида затухает и теоретически, и практически

Посмотрим на практике, загрузим данные, создадим сеть, обучим ее и посмотрим, как проходит обучение. 

Загрузим датасет MNIST:

In [ ]:
import torchvision
import torchvision.transforms as transforms
from IPython.display import clear_output
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# transforms for data
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.13), (0.3))]
)

train_set = MNIST(root="./MNIST", train=True, download=True, transform=transform)
test_set = MNIST(root="./MNIST", train=False, download=True, transform=transform)

batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

clear_output()
print("Already downloaded!")

Создадим сеть с сигмоидой в качестве функции активации:

In [ ]:
import torch.nn as nn


class SimpleMNIST_NN(nn.Module):
    def __init__(self, n_layers, activation=nn.Sigmoid):
        super().__init__()
        self.n_layers = n_layers  # Num of layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation]  # input layer
        for _ in range(n_layers - 1):  # append num of layers
            layers.append(nn.Linear(100, 100))
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10))  # 10 classes
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # reshape to [-1, 784]
        x = self.layers(x)
        return x

Код для визуализации результатов обучения моделей:

In [ ]:
def exponential_smoothing(scalars, weight):
    last = scalars[0]
    smoothed = []
    for point in scalars:
        smoothed_val = last * weight + (1 - weight) * point
        smoothed.append(smoothed_val)
        last = smoothed_val

    return smoothed

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_history(history, num_epochs=5, smooth_val=0.90):
    fig, ax = plt.subplots(3, 1, figsize=(12, 14))
    for stage_idx, (stage_lbl, stage_title) in enumerate(
        zip(["loss_on_train", "loss_on_test"], ["train loss", "test loss"])
    ):
        # plot history on each learning step
        epoch_len = len(history[stage_lbl]) // num_epochs
        full_stage_len = len(history[stage_lbl])
        ax[stage_idx].plot(
            exponential_smoothing(history[stage_lbl], smooth_val),
            label="smoothed",
            color="m",
        )
        ax[stage_idx].plot(history[stage_lbl], label="raw", alpha=0.2, color="c")
        ax[stage_idx].set_title(stage_title)
        ax[stage_idx].set_xlabel("epochs")
        ax[stage_idx].set_ylabel("loss")
        epochs_ticks_positions = np.arange(stop=full_stage_len + 1, step=epoch_len)
        ax[stage_idx].set_xticks(epochs_ticks_positions)
        ax[stage_idx].set_xticklabels(np.arange(num_epochs + 1))
        ax[stage_idx].legend()

        # plot mean train and test loss combined
        mean_loss_on_epoch = [
            np.mean(history[stage_lbl][i : i + epoch_len])
            for i in range(0, full_stage_len, epoch_len)
        ]
        std_loss_on_epoch = [
            np.std(history[stage_lbl][i : i + epoch_len])
            for i in range(0, full_stage_len, epoch_len)
        ]

        ax[2].set_title("\nAverage loss per epoch")
        ax[2].errorbar(
            np.arange(num_epochs) + stage_idx / 30.0,
            mean_loss_on_epoch,
            yerr=std_loss_on_epoch,
            capsize=5,
            fmt="X--",
            label=stage_title,
        )
        ax[2].set_xticks(np.arange(5))
        ax[2].set_xticklabels(np.arange(5))
        ax[2].set_xlabel("epochs")
        ax[2].set_ylabel("loss")
        ax[2].legend()

    fig.suptitle(history["model_name"], fontsize=24)
    plt.show()

Функция для обучения сети на обучающей выборке:

In [ ]:
import torch

# compute on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train_epoch(model, optimizer, criterion, train_loader):
    loss_history = []
    for batch in train_loader:
        optimizer.zero_grad()
        x_train, y_train = batch  # parse data
        x_train, y_train = x_train.to(device), y_train.to(device)  # compute on gpu
        y_pred = model(x_train)  # get predictions
        loss = criterion(y_pred, y_train)  # compute loss
        loss_history.append(loss.cpu().detach().numpy())  # write loss to log
        loss.backward()
        optimizer.step()
    return loss_history

Функция для валидации сети на валидационной выборке:

In [ ]:
def validate(model, criterion, val_loader):
    cumloss = 0
    loss_history = []
    with torch.no_grad():
        for batch in val_loader:
            x_train, y_train = batch  # parse data
            x_train, y_train = x_train.to(device), y_train.to(device)  # compute on gpu
            y_pred = model(x_train)  # get predictions
            loss = criterion(y_pred, y_train)  # compute loss
            loss_history.append(loss.cpu().detach().numpy())  # write loss to log
            cumloss += loss
    return cumloss / len(val_loader), loss_history  # mean loss and history

Функция для обучения модели:

In [ ]:
from tqdm.notebook import tqdm


def train_model(model, optimizer, model_name=None, num_epochs=5):

    criterion = nn.CrossEntropyLoss().to(device)

    train_history = {}
    train_history["model_name"] = model_name
    train_history["loss_on_train"] = []
    train_history["loss_on_test"] = []

    for epoch in tqdm(range(num_epochs)):
        loss_on_train = train_epoch(model, optimizer, criterion, train_loader)
        _, loss_on_test = validate(model, criterion, test_loader)
        train_history["loss_on_train"].extend(loss_on_train)
        train_history["loss_on_test"].extend(loss_on_test)
    return train_history

Создадим и запустим обучение модели с двумя слоями:

In [ ]:
import torch.optim as optim

model = SimpleMNIST_NN(n_layers=2).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

history = train_model(model, optimizer, model_name="n_layers2_sigmoid")
plot_history(history)

In [ ]:
model = SimpleMNIST_NN(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)
history = train_model(model, optimizer, model_name="n_layers3_sigmoid")
plot_history(history)

Нейросеть с тремя слоями вообще не учится. Почему? Можем попробовать разобраться.

Для этого напишем функции, которые будут следить за распределением градиентов и активаций на наших слоях.

Воспользуемся методом `register_backward_hook` библиотеки PyTorch для того, чтобы выполнять эти функции при каждом пропускании градиента.

In [ ]:
from collections import defaultdict


def get_forward_hook(history_dict, key):
    def forward_hook(self, input_, output):
        history_dict[key] = input_[0].cpu().detach().numpy().flatten()

    return forward_hook


def get_backward_hook(history_dict, key):
    def backward_hook(grad):  # for tensors
        history_dict[key] = grad.abs().cpu().detach().numpy().flatten()

    return backward_hook


def register_model_hooks(model):
    cur_ind = 0
    hooks_data_history = defaultdict(list)
    for child in model.layers.children():
        if isinstance(child, nn.Linear):
            cur_ind += 1
            forward_hook = get_forward_hook(hooks_data_history, f"activation_{cur_ind}")
            child.register_forward_hook(forward_hook)

            backward_hook = get_backward_hook(hooks_data_history, f"gradient_{cur_ind}")
            child.weight.register_hook(backward_hook)
    return hooks_data_history

Запустим обучение модели с 3 слоями:

In [ ]:
model = SimpleMNIST_NN(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

hooks_data_history = register_model_hooks(model)

history = train_model(model, optimizer, model_name="n_layers3_sigmoid2")

In [ ]:
def plot_hooks_data(hooks_data_history):
    keys = hooks_data_history.keys()
    n_layers = len(keys) // 2

    activation_names = [f"activation_{i + 1}" for i in range(1, n_layers)]
    activations_on_layers = [
        hooks_data_history[activation] for activation in activation_names
    ]

    gradient_names = [f"gradient_{i + 1}" for i in range(n_layers)]
    gradients_on_layers = [hooks_data_history[gradient] for gradient in gradient_names]

    for plot_name, values, labels in zip(
        ["activations", "gradients"],
        [activations_on_layers, gradients_on_layers],
        [activation_names, gradient_names],
    ):
        fig, ax = plt.subplots(1, len(labels), figsize=(14, 4), sharey="row") 
        for label_idx, label in enumerate(labels):
            ax[label_idx].boxplot(values[label_idx], labels=[label])
        plt.show()

In [ ]:
plot_hooks_data(hooks_data_history)

Мы видим, что градиент нашей модели стремительно затухает. Первые слои (до которых градиент доходит последним), получают значения градиента, мало отличимые от нуля.

Причем, это будет верно с самых первых шагов обучения нашей модели.

Это явление получило название **паралич сети**



# Затухание градиента

Откуда оно берется? 

Посмотрим на обычную сигмоиду

$$\sigma(z) = \dfrac 1 {1 + e^{-z}}$$

Ее производная, как мы уже выводили, равна

$$\dfrac {\partial \sigma(z)} {\partial z} = \sigma(z) (1 - \sigma(z))$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L05/out/activation_function_sigmoid.png" width="1000">

Какое максимальное значение у такой функции?

Сигмоида находится в пределах от 0 до 1. Максимальное значение производной по сигмоиде  $=\dfrac 1 4$

Теперь возьмем простую нейронную сеть:


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/simple_nn_with_sigmoid.png" width="750">

Посчитаем у нее градиент

$$\dfrac {\partial L} {\partial z_4} = \dfrac {\partial L} {\partial y} \dfrac {\partial y} {\partial z_4} = \dfrac {\partial L} {\partial y} \dfrac {\partial \sigma(w_5z)} {\partial z} w_5 \le \dfrac 1 4 \dfrac {\partial L} {\partial y}  w_5 $$

Аналогично можно посчитать градиент для $z_3$

$$\dfrac {\partial L} {\partial z_3} = \dfrac {\partial L} {\partial z_4} \dfrac {\partial z_4} {\partial z_3} \le \dfrac {\partial L} {\partial y} \dfrac {\partial \sigma(w_4z)} {\partial z} w_5 \le \left({\dfrac 1 4}\right)^2 \dfrac {\partial L} {\partial y}  w_5 w_4$$

И так далее

$$\dfrac {\partial L} {\partial x}  \le \left({\dfrac 1 4}\right)^5 \dfrac {\partial L} {\partial y}  w_5 w_4 w_3 w_2 w_1$$

Таким образом, градиент начинает экспоненциально затухать, если веса маленькие.

Если веса большие, то градиент наоборот начнет экспоненциально возрастать (взрыв градиента).

Для некоторых функций активации картина будет не столь катастрофична, но тоже неприятна.

При выполнении заданий вы посмотрите, например, как ведет себя функция ReLU в этом случае.

# Инициализация весов

Одним из способов борьбы с затухающим градиентом является правильная инициализация весов. Как это сделать?

**Идея 1:** инициализировать все веса константой. 

Проблема: градиент по всем весам будет одинаков, как и обновление весов. Все нейроны в слое будут учить одно и то же, или, в случае $const = 0$, [не будут учиться вообще](https://habr.com/ru/post/592711/).

Вывод: в качестве начальных весов нужно выбирать различные значения.

**Идея 2:** инициализировать веса нормальным (Гауссовским) шумом с матожиданием 0 и маленькой дисперсией. 

Маленькая дисперсия нужна, чтобы не получить огромные градиенты за большие изначальные ошибки в предсказании.

In [ ]:
# Normal distribution: mu = 0, sigma = 1

x = np.arange(-4, 4.1, 0.1)
y = np.exp(-np.square(x) / 2) / np.sqrt(2 * np.pi)

plt.style.use("seaborn-whitegrid")
plt.title("Normal distribution: mu = 0, sigma = 1", size=15)
plt.plot(x, y)
plt.show()

Проблема: инициализация нормальным шумом не гарантирует отсутствие взрыва или затухания градиета.

**Идея 3:** формализуем условия, при которых не будет происходить взрыв градиентов.

Нам бы хотелось, чтобы 
дисперсии признаков, получаемых на каждом слое, были одинаковы: 

$$Dz^i = Dz^j. \tag{1}$$

И чтобы  начальные дисперсии градиентов для разных слоев были одинаковы:
 
$$D\dfrac {\partial L} {\partial z^i} = D\dfrac {\partial L} {\partial z^j}. \tag{2}$$

При выполнении этих условий градиент не затухает и не взрывается.

Попытаемся выполнить эти условия.

## Инициализация Ксавье (Xavier Glorot)

### Вывод Xavier

Рассмотрим функцию активации гиперболический тангенс (Tanh).

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/simple_nn_with_tanh.png" width="600">

Это — [нечетная функция](https://ru.wikipedia.org/wiki/%D0%A7%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D1%8C_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8) с единичной производной в нуле. Функция и ее производная изображены ниже.

In [ ]:
x = np.arange(-10, 10.1, 0.1)
y = np.tanh(x)
dy = 1 / np.cosh(x)

plt.style.use('seaborn-whitegrid')
fig, (im1, im2) = plt.subplots(2, 1, figsize=(7, 7))
im1.set(title = "tanh(x)")
im1.plot(x[0:51], y[0:51], 'r', x[50:96], y[50:96], 'b', 
         x[95:106], y[95:106], 'g', x[105:151], y[105:151], 'b', 
         x[150:201], y[150:201], 'r') 
im2.set(title = "tanh'(x)")
im2.plot(x[0:51], dy[0:51], 'r', x[50:96], dy[50:96], 'b',
         x[95:106], dy[95:106], 'g', x[105:151], dy[105:151], 'b',
         x[150:201], dy[150:201], 'r')
plt.show()

При выборе весов нам важно не попасть в красные зоны с почти нулевой производной, т.к. в этих областях градиент затухает. Мы хотим инициализировать веса таким образом, чтобы признаки, поступающие на слой активации, находились в зеленой области в окрестности нуля. Матожидание признаков, поступающих на слой активации, будет равно нулю
$$E(z^i_t w_{kt})=0 \tag{3}.$$



Выход слоя активации на текущем слое будет зависеть от выхода слоя активации на предыдущем: $$z^{i+1} = f(z^iW^i). \tag{4}$$

В окрестности нуля функцию гиперболический тангенс можно считать [линейной](https://ru.wikipedia.org/wiki/%D0%A0%D1%8F%D0%B4_%D0%A2%D0%B5%D0%B9%D0%BB%D0%BE%D1%80%D0%B0):
$$z^{i+1} \approx z^i W^i \tag{5}.$$
Матожидание выхода функции активации также равно нулю:
$$E(z^i_t)=0 \tag{6}$$

Значения весов в слоях будут генерироваться генератором случайных чисел, поэтому веса в различных слоях можно считать независимыми случайными величинами. Для независимых случайных величин выполняются формулы: 
1. Дисперсии суммы двух независимых величин:

$$D(\eta + \gamma) = D\eta + D\gamma \tag{7}.$$

2. Дисперсии произведения двух независимых величин:

$$D\eta\gamma = E(\eta\gamma)^2 - (E\eta\gamma)^2 = E\eta^2E\gamma^2 - (E\eta)^2(E\gamma)^2 \tag{8}.$$ 


Распишем условие $(1)$ с использованием $(5)$ и $(7)$:

$$D(z^{i+1}_{k}) = D(\sum_t z^i_t w_{kt}) = \sum_t D(z^i_t w_{kt}) \tag{9}$$

Дисперсии признаков на входе функции активации берем одинаковые:

$$D(z^{i+1}_{k}) = n D(z^i_0 w_{k0}),\tag{10}$$

где $n$ — размерность выхода слоя.

Применяем нашу формулу $(8)$ и получаем:

$$D(z^{i+1}_{k}) = n [E(z^i_0)^2E(w_{k0})^2 - (Ez^i_0)^2(Ew_{k0})^2]. \tag{11}$$

Используем $(6)$:

$$D(z^{i+1}_{k}) =   n E(z^i_0)^2E(w_{k0})^2. \tag{12}$$

Матожидание выходов активаций и весов равны 0. Из этого: 

$$D(z^{i}_{0}) = E(z^{i}_{0})^2 - (Ez^{i}_{0})^2 = E(z^{i}_{0})^2, \tag{13}$$

$$D(w_{k0}) = E(w_{k0})^2 - (Ew_{k0})^2 = E(w_{k0})^2.$$


Подставляем в $(12)$:

$$D(z^{i+1}_{k}) = n D(z^i_0)D(w_{k0}). \tag{14}$$

Из $(14)$ следует формула для зависимости выхода активаций любого слоя от весов предыдущих слоев и дисперсии исходных данных:

$$Dz^i = Dx \prod_{p=0}^{i-1}n_pDW^p, \tag{15}$$

где $n_p$ — размерность выхода слоя p-го слоя.

Аналогично можно вывести формулу для градиентов по активациям:

$$D(\dfrac {\partial L} {\partial z^i}) = D(\dfrac {\partial L} {\partial z^d} ) \prod_{p=i}^{d}n_{p+1}DW^p. \tag{16}$$

Вспоминаем условия $(1)$, $(2)$:

$$Dz^i = Dz^j \tag{1},$$

$$D\dfrac {\partial L} {\partial z^i} = D\dfrac {\partial L} {\partial z^j} \tag{2}.$$

С учетом $(15)$, $(16)$ они эквивалентны условиям:

$$n_iDW^i = 1 \tag{17}$$

$$n_{i+1}DW^i = 1 \tag{18}$$

Условия могут быть невыполнимы одновременно:

 $$n_i \ne n_{i+1}. $$

Возьмем компромисс — среднее гармоническое решений первого и второго уравнения:

$$DW^i = \dfrac 2 {n_i + n_{i+1}} \tag{19}.$$

**Итого:** нам нужно инициализировать веса нейронов случайными величинами со следующими матожиданием и дисперсией:

$$ EW^i = 0,$$

$$DW^i = \dfrac 2 {n_i + n_{i+1}}.$$

При инициализации Xavier используется [равномерное распределение](https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D0%BF%D1%80%D0%B5%D1%80%D1%8B%D0%B2%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D0%B2%D0%BD%D0%BE%D0%BC%D0%B5%D1%80%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5): 

$$W_i \sim U[a, b ],$$

где $a=-b$, так как матожидание равно 0. 

Дисперсия которого выражается формулой: 
$$D(U[a, b]) = \dfrac 1 {12} (b -a)^2 = \dfrac 4 {12} b^2 = \dfrac 1 {3} b^2.$$

Из $(19)$ получим:

$$ b = \sqrt{\dfrac {6} {n_i + n_{i + 1}}}$$

**Инициализация  Xavier** — это инициализация весов случайной величиной с распределением:

$$W_i \sim U[-\sqrt{\dfrac {6} {n_i + n_{i + 1}}}, \sqrt{\dfrac {6} {n_i + n_{i + 1}}}],$$ 

где $n_i$ — размерность выхода слоя n-го слоя.



Чтобы понять, что происходит с выходами слоя активации при использовании инициализации Xavier, рассмотрим картинку из оригинальной статьи [Xavier, Yoshua, "Understanding the difficulty of training deep feedforward neural networks", Aistats, 2010](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf):

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/xavier_procentile_and_deviation_with_and_without_init.png" width="600">

На картинке изображена зависимость 98-[процентиля](https://en.wikipedia.org/wiki/Percentile) (отдельные маркеры) и стандартного отклонения (соединенные маркеры) значений на выходе слоя активации $tanh$ от эпохи обучения для различных слоев нейросети. 

Верхнее изображение — инициализация весов с помощью нормального распределения $W_i \sim U[-\dfrac {1} {\sqrt{n_i}}, \dfrac {1} {\sqrt{n_i}} ]$

Нижнее с использованием инициализации Xavier.

На верхнем изображении видно, как значения 98-процентиля уходят в значения +1 и -1 (сначала на выходе первого слоя, потом на выходе второго и т.д.). Это значит, что для части нейронов происходит затухание градиентов (они переходят в область, отмеченную на графиках $tanh(x)$, $tanh’(x)$ красным). На нижней картинке такого не происходит. 

## Инициализация Каймин Хе (Kaiming He)

Для функции активации ReLU и ее модификаций (PReLU, Leaky ReLU и т.д.) аналогично инициализации Xavier можно расписать условия $(1)$, $(2)$. Так вводится He-инициализация. 

### Вывод Kaiming

Аналогично выводу для Xavier получаем выражения:

$$Dz^i = Dx \prod_{p=0}^{i-1}\dfrac 1 2 n_pDW^p $$

$$D(\dfrac {\partial L} {\partial z^i}) = D(\dfrac {\partial L} {\partial z^d} ) \prod_{p=i}^{d}\dfrac 1 2 n_{p+1}DW^p $$

где $n_p$ — размерность выхода слоя $p$-го слоя.

Условия $(1)$, $(2)$ эквивалентны условиям: 

$$  \dfrac {n_iDW^i} {2}  = 1, $$

$$\dfrac {n_{i+1}DW^i} {2} = 1.$$

Можно опять взять среднее гармоническое. Но на практике берут либо $ \frac 2 {n_i}$, либо $\frac 2 {n_i + 1}$

Итого получим:

$$W^i \sim N(0, sd=\sqrt{\frac 2 n_i})$$

Более подробно с выводом инициализации Каймин Хе можно ознакомиться в оригинальной [статье](https://arxiv.org/pdf/1502.01852v1.pdf).

## Важность инициализации весов

1. Нейросеть может сойтись значительно быстрее


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/weight_initialization_influence_convergence_neural_networks.png" width="550">

<center><p><em>Source: <a href="https://arxiv.org/pdf/1502.01852v1.pdf">Delving Deep into Rectifiers:
Surpassing Human-Level Performance on ImageNet Classification</a></p> </em></center>

2. В зависимости от выбранной активации сеть вообще может сойтись или не сойтись

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/activation_function_influence_convergence_neural_networks.png" width="550">

<center><p><em>Source: <a href="https://arxiv.org/pdf/1502.01852v1.pdf">Delving Deep into Rectifiers:
Surpassing Human-Level Performance on ImageNet Classification</a></p> </em></center>

## Обобщение инициализаций Ксавье и Каймин Хе

Вообще говоря, коэффициенты в инициализациях (числитель в формуле для дисперсии), зависят от конкретной выбранной функции активации.
В PyTorch есть [функции](https://pytorch.org/docs/stable/nn.init.html) для вычисления этих коэффициентов.


## Ортогональная инициализация

Также иногда используется так называемая ортогональная инициализация весов. 
Для каждого слоя мы убеждаемся, что изначальная матрица весов является ортогональной. 

Выберем ортогональную матрицу весов 
$$W: WW^T = 1$$

Тогда:
1.  норма активации сохраняется (опять же, активации между слоями остаются в одном масштабе)
$$||s_{i+1}|| = ||W_{i}s_i|| = ||s_i||$$

2.  все нейроны делают «разные» преобразования
$$ ⟨W_i, W_j⟩ = 0~i \ne j$$
$$ ⟨W_i, W_j⟩ = 1~i = j$$


Иногда такая инициализация обеспечивает значительно лучшую сходимость, [тут](https://arxiv.org/pdf/1312.6120.pdf) можно почитать об этом подробнее.

## Инициализация весов в PyTorch

Для инициализации весов PyTorch используется модуль `torch.nn.init`

В нем определены разные функции для инициализации весов.

Нюанс состоит в том, что обычно для слоев разного типа может требоваться разная инициализация. Поэтому в функции, которая инициализирует слои вашей нейронной сети, желательно прописывать разное поведение для разных слоев.

Попробуем, например, добавить в нашу нейросеть инициализацию. Нам нужна инициализация Xavier, так как у нас `nn.Sigmoid`

Метод `torch.nn.init.calculate_gain` возвращает рекомендуемое значение коэффициента масштабирования для стандартного отклонения заданной функции активации.

In [ ]:
class SimpleMNIST_NN(nn.Module):
    def __init__(self, n_layers, activation=nn.Sigmoid, init_form="normal"):
        super().__init__()
        self.n_layers = n_layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100))
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init_form = init_form
        if self.init_form is not None:
            self.init()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        return x

    # xavier weight initialization
    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                if self.init_form == "normal":
                    torch.nn.init.xavier_normal_(child.weight, gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                elif self.init_form == "uniform":
                    torch.nn.init.xavier_uniform_(child.weight, gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                else:
                    raise NotImplementedError()

Запустим обучение модели с инициализацией весов Xavier:

In [ ]:
model = SimpleMNIST_NN(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

# plotting weights values of first(input layer)
plt.figure(figsize=(15, 5))
plt.hist(
    list(model.layers.children())[0].weight.cpu().detach().numpy().reshape(-1), bins=100
)
plt.title("weights histogram")
plt.xlabel("values")
plt.ylabel("counts")
plt.show()

history = train_model(model, optimizer, model_name="n3_layers_sigmoid_havier")

In [ ]:
plot_history(history)

Видим, что нейросеть стала хоть как-то учиться.

# Нормализация

## Нормализация входных данных

Представим себе, что данные, которые мы подаем в нейросеть, распределены следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/data_before_normalization.png" width="400">

Фактически нейросети работают со скалярными произведениями. В этом плане два вектора, изображенных на рисунке, не сильно отличаются. Также и точки нашего датасета слабо разделимы. Чтобы с этим работать, нейросеть сначала должна подобрать удобное преобразование, а затем только сравнивать наши объекты. Понятно, что это усложняют задачу. 

Для того, чтобы облегчить нейросети задачу, входные признаки часто нормируют:

$$x1' = \dfrac {x1 - \mu_{x1}} {\sigma_{x1}}$$
$$x2' = \dfrac {x2 - \mu_{x2}} {\sigma_{x2}}$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/data_after_normalization.png" width="450">


 Такое преобразование действительно помогает нейросети 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/normalization_helps_find_minimum_of_function.png" >


## Covariate shift (Ковариантный сдвиг)

**Covariate shift** &mdash; явление, когда признаки тренировочной и тестовой выборок **распределены по-разному**. Ковариантный сдвиг может стать серьезной проблемой для практического применения моделей.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/covariate_shift.png" width="300"><\center>

Модель учиться сопоставлять целевые значения признакам. В такой ситуации модель не в состоянии делать адекватные предсказания на тесте, так как во время обучения она не видела области пространства, в которой расположены тестовые объекты.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/covariate_shift_problem.png" width="300"></center>

Выделяют **два источника ошибок**, приводящих к **ковариантному сдвигу**:

1. **Систематические ошибки**:
*   *Ошибки при сборе данных* (предвзятый метод сбора данных, нерепрезентативная выборка).

**Пример:** стандартные наборы данных для задачи **Face Recognition** не сбалансированы по полу, возрасту и этнической принадлежности, поэтому обученные на них модели могут плохо работать с редкими классами. Это привело к обвинениям таких корпораций, как Microsoft, IBM и Amazon в [расизме](https://sitn.hms.harvard.edu/flash/2020/racial-discrimination-in-face-recognition-technology/).

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/face_recognition_racism.webp" width="1000"></center>
<center><p><em>Source: <a href="https://sitn.hms.harvard.edu/flash/2020/racial-discrimination-in-face-recognition-technology/">Racial Discrimination in Face Recognition Technology</a></p> </em></center>

**Что делать?**
Следить за репрезентативностью данных, использовать Importance Reweighting (при обучении давать больший вес редким объектам). 


*  *Различие условий сбора данных для train и test выборки*

**Пример:** Задачи компьютерного зрения. Данных для обучения может быть недостаточно, поэтому часто для обучения используют датасеты из Интернета. При этом условия съемки (качество, разрешение, освещенность) train и test выборки могут отличаться.

**Что делать?** Аугментировать данные с учетом условий применения.




* *Ошибки предобработки данных*

**Пример:** В картах пациентов больницы А, использованных для train, рост пациента указан метрах, в картах пациентов больницы Б, использованных для test, рост пациента указан в футах. При составлении датасета данные не привели к одной **единице измерения**.


**Что делать?** Искать ошибку.

2. **Нестационарная среда**

При обучении мы используем ограниченный набор признаков и объектов. При этом есть факторы, которые мы можем не учесть: износ оборудования, изменение политических условий и т.д. что может приводить к постепенной или резкой деградации модели при применении ее в реальных условиях.

**Что делать?** 
* Удалять из модели маловажные нестационарные признаки
* Дообучать модель на новых данных
* [Компенсировать сдвиг](https://www.researchgate.net/publication/339021786_Covariate_Shift_A_Review_and_Analysis_on_Classifiers)

**Замечание:** Просадка качества на test выборке может быть также связана с **проблемами обобщающей способности модели**. Причины могут быть разными, например переобучение под validate выборку или отсутствие связи между признаками и целевым значением (задача в данной постановке не решается).

**Практический совет:** для быстрого обнаружение ковариантного сдвига можно **обучить модель**, которая будет предсказывать, относится ли объект к **train** или **test выборке**. Если модель легко делит данные - имеет смысл визуализировать значения признаков, по которым она это делает.


## Internal covariate shift

В статье [“Batch Normalization: Accelerating Deep Network Training b y Reducing Internal Covariate Shift”](https://arxiv.org/pdf/1502.03167.pdf) 2015 года авторы предположили, что похожее явление имеет место внутри нейросети, назвав его **internal covariate shift**.

**Internal covariate shift** - это изменение распределения выхода слоя активации из-за изменения обучаемых параметров во время обучения.

Пусть у нас $i$-й слой переводит выход $i$-1 слоя с распределением <font color='#F9B041'>$f_{i-1}(x)$</font> в новое пространство с распределением <font color='#5D5DA6'>$f_{i}(x)$</font>. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/internal_covariate_shift_example.png" width="1000">

При обучении:
- Нейросеть делает предсказание, 
- Считается значение функции потерь,
- Делается обратное распространение ошибки,
- Обновляются веса.


После обновления весов $i$-й слой будет переводить выход $i$-1 слоя <font color='#F9B041'>$f_{i-1}(x)$</font> в пространство с другим распределением <font color='#5D5DA6'>$f^*_{i}(x)$</font>. 

При этом $i$+1 слой учился работать со старым распределением <font color='#5D5DA6'>$f_{i}(x)$</font> и будет хуже обрабатывать <font color='#5D5DA6'>$f^*_{i}(x)$</font>.

### Плохой вариант борьбы с этим

Давайте на каждом слое просто нормировать каждый признак, используя среднее и дисперсию по батчу

$$ \hat{x}_{i} = \frac{x_{i} - \mu_{B}}{\sigma_{B} + \epsilon}$$




Проблема в том, что таким образом мы можем попасть в область линейной составляющей нашей функции. Например, в случае сигмоиды 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/domain_of_linear_of_sigmoid_function.png" width="500">

Получаем набор линейных слоев фактически без функций активации, следовательно, все вырождается в однослойную сеть. Не то, что нам надо.

## Batch Normalization

Нам надо дать нейронной сети возможность перемещать распределение выходов слоя из области 0 и самой подбирать дисперсию. Для этой цели используется **батч-нормализация** (*batch normalization*), которая вводит в нейронную сеть дополнительную операцию между соседними скрытыми слоями. Она состоит из нормализации входящих (в слой батч-нормализации) значений, полученных от одного скрытого слоя, масштабирования и сдвига с применением двух новых параметров и передачи полученных значений на вход следующему скрытому слою.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization.png" width="800">

Параметры, используемые в батч-нормализации ($\gamma$ — отвечающий за сжатие и $\beta$ — отвечающий за сдвиг), являются обучаемыми параметрами (наподобие весов и смещений скрытых слоев). 

Помимо обучаемых параметров $\gamma$ и $\beta$ в слое батч-нормализации существуют также необучаемые параметры: __скользящее среднее__ средних (_Mean Moving Average_) и скользящее среднее дисперсий (_Variance Moving Average_), служащие для сохранения состояния слоя батч-нормализации. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnorm_layer_parameters.png"> 

Параметры $\gamma$, $\beta$, а также оба скользящих средних вычисляются для каждого слоя батч-нормализации отдельно и являются векторами с длиной, равной количеству входящих признаков.

В процессе обучения мы подаем в нейронную сеть по одному мини-батчу за раз. Процедуру обработки значений $x$ из одного мини-батча $ B = \{x_{1},\ldots, x_{m}\} $ можно представить следующим образом:



<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batch_normalization_compute_moving_average.png" width="1000">

Шаг масштабирования и сдвига является главным новшеством батч-нормализации, поскольку в отличие от предыдущего рассмотренного примера, нормированные значения больше не обязаны иметь среднее равное 0 и единичную дисперсию. Батч-нормализации позволяет сдвигать среднее нормированных значений и масштабировать дисперсию. Фактически, теперь нейросеть даже может отменить нормализацию входных данных, если считает ее ненужной.

Для наглядности проиллюстрируем размерности промежуточных переменных на следующем изображении:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batch_normalization_compute_moving_average_scheme.png" width="1000">

После прямого прохода параметры $\gamma$ и $\beta$ обновляются через обратное распространение ошибки так же, как и веса скрытых слоев.

### Скользящее среднее

Выше мы обсуждали то, что в процессе обучения слой  батч-нормализации рассчитывает значение среднего и дисперсии каждого признака в соответствующем мини-батче. Однако во время предсказания батч у нас уже отсутствует &mdash; откуда брать среднее и дисперсию? 

Во время предсказания используется скользящее среднее, которое было рассчитано и сохранено в процессе обучения. 

$$ \mu_{mov_{B}} = (1-\alpha)\mu_{mov_{B-1}}+\alpha\mu_{B} $$

$$ \sigma_{mov_{B}} = (1-\alpha)\sigma_{mov_{B-1}}+\alpha\sigma_{B} $$

Обычно используется параметр $\alpha = 0.1$

Почему используется именно скользящее среднее, а не, например, статистика всей обучающей выборки? Дело в том, что при таком подходе нам бы пришлось хранить средние всех признаков для всех батчей, пропущенных через нейросеть в ходе обучения. Это ужасно невыгодно по памяти. Вместо этого скользящее среднее выступает в качестве приближенной оценки среднего и дисперсии обучающего набора. В этом случае эффективность использования ресурсов увеличивается: поскольку вычисления производятся постепенно, нам нужно хранить в памяти только одно число — значение скользящего среднего, полученное на последнем шаге.

Проиллюстрировать преимущество использования скользящего среднего можно на следующем примере:

Предположим, что у нас есть некоторая генеральная совокупность объектов, обладающих некоторым признаком $x$, (соответствующая абстрактной обучающей выборке), и некоторый черный ящик, извлекающий по $k$ объектов из этой генеральной совокупности (соответствующий абстрактному даталоадеру). Наша задача — дать оценку ожидаемому среднему этих $k$ объектов. При этом ожидаемое среднее для $k$ объектов может отличаться от среднего генеральной совокупности, поскольку могут накладываться дополнительные условия на извлекаемую выборку. В данном примере для простоты будем извлекать $k$ объектов из некого распределения случайным образом:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

k = 500  # sample size
n = 2
p = 0.5

sample = np.random.negative_binomial(n, p, k)
sns.histplot(data=sample, discrete=True)
plt.show()

Оценить ожидаемое среднее теоретически, не зная, как распределен признак $x$ наших объектов, трудно. Мы можем собрать большое количество средних и произвести оценку с их помощью, но для этого нам потребуется хранить в памяти все эти значения, что опять приведет к неэффективному расходу ресурсов. Более эффективным решением будет воспользоваться скользящим средним. Давайте сравним эти два метода:

In [ ]:
ema = 0
alpha = 0.01
means = np.array([])

for i in range(10000):
    sample = np.random.negative_binomial(n, p, 50)
    ema = (1 - alpha) * ema + alpha * sample.mean()
    means = np.append(means, sample.mean())

Посчитаем количество памяти, затрачиваемое на хранение списка средних значений признака $x$ по выборкам из $k$ объектов, и количество памяти, затрачиваемое на хранение скользящего среднего:

In [ ]:
import sys

print(f"{sys.getsizeof(ema)} bytes")

Количество памяти для хранения списка средних:

In [ ]:
print(f"{sys.getsizeof(means)} bytes")

Видно, что на хранение массива средних значений расходуется на порядки больше памяти, чем на хранение одного скользящего среднего. Теперь давайте воспользуемся тем, что мы сэмплировали случайные выборки из известного распределения, и можем теоретически рассчитать их среднее. В нашем примере мы извлекали выборки из негативного биномиального распределения с параметрами $n=2$ и $p=0.5$, для которого среднее рассчитывается по формуле $mean=\frac{np}{1-p}=2$. Мы знаем, что при достаточно большом количестве сэмплированных выборок среднее распределения выборочных средних будет стремиться к среднему генеральной совокупности. Сравним результаты, полученные с использованием сохраненных выборочных средних и скользящего среднего с теоретическим расчетом:

Среднее признака $x$ по $k$ объектам, оцененное с помощью скользящего среднего:

In [ ]:
print(f"{ema:.8f}")

Среднее признака $x$ по $k$ объектам, оцененное по всем сэмплированным выборкам:

In [ ]:
print(f"{means.mean():.8f}")

Видно, что мы получили довольно точную оценку, использовав скользящее среднее.

### Защита от нулей в знаменателе

Чтобы у нас не мог возникнуть 0 в знаменателе, добавляем маленькое число — $\epsilon$. Например, равное 1e-5


$$ \hat{x}_{i} = \frac{x_{i} - \mu_{B}}{\sigma_{B} + \epsilon}$$

$$ BN_{\gamma, \beta}(x_{i}) = \gamma \hat{x}_{i} + \beta $$

### Линейный слои и конволюции

Свёрточный можно свести к линейному слою с очень жесткими ограничениями на веса. Поэтому неудивительно, что BatchNorm можно применять и для линейных слоев, и для свёрточных. 

Со сверточным слоем есть единственный нюанс — у нас "одним признаком" считается вся получаемая **feature map**. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/feature_map.png" width="500">

И нормализация идет по всей такой feature map (по всему каналу) для всех объектов. 

### Пример работы

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnorm_efficiency.png" width="550">

<center><p><em>Source: <a href="https://arxiv.org/pdf/1502.03167.pdf">Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shiftn</a></p> </em></center>



Этот метод действительно работает. 
Видим, что нейросети с батч-нормализацией:

1. Сходятся быстрее, чем нейросети без неё
2. Могут работать с более высоким начальным значением learning rate, причем это позволяет достигать лучших результатов
3. BatchNorm позволяет глубокой нейросети работать даже с функцией активации в виде сигмоиды. Без него такая сеть не обучилась бы вовсе. 

### Градиент

Вычисление градиента batchnorm &mdash; интересное упражнение на понимание того, как работает backpropagation. В лекции мы это опускаем, можете ознакомиться самостоятельно:

[Deriving the Gradient for the Backward Pass of Batch Normalization](https://kevinzakka.github.io/2016/09/14/batch_normalization/)



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnorm_gradient.png" width="700">

<center><p><em>Source: <a href="https://arxiv.org/pdf/1502.03167.pdf">Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shiftn</a></p> </em></center>



### Batch Normalization как регуляризация

Почему для нейросети с батчнормализацией можно использовать более высокие learning rate? 

Оказывается, батч-нормализация делает неявную регуляризацию на веса.

Допустим, мы решили увеличить веса в $a$ раз.

Так как мы шкалируем, то домножение весов $W$ на константу выходных значений слоя не меняет

$$BN((aW)u) = BN(Wu)$$

Градиент слоя по входу не меняется

$$\dfrac {\partial BN((aW)u)} {\partial u} = \dfrac {\partial BN(Wu)} {\partial u}$$

А градиент по весам уменьшается в $a$ раз

$$\dfrac {\partial BN((aW)u)} {\partial aW} = \dfrac 1 a \dfrac {\partial BN(Wu)} {\partial W} $$

Таким образом, нейросеть автоматически не дает большим весам расти

### Сглаживающий эффект Batch Normalization

**Batch Normalization** была разработана на идее необходимости коррекции **Internal covariate shift**. В 2019 году вышла статья [How Does Batch Normalization Help Optimization?](https://arxiv.org/pdf/1805.11604.pdf), которая показала, что влияние коррекции **Internal covariate shift** на качество обучения не так велико, как считали авторы **Batch Normalization**.

Другим интересным эффектом Batch Normalization оказалось **сглаживание ландшафта** функции потерь. Batch Normalization улучшает гладкость пространства решений и облегчает поиск в нем минимума. Именно благодаря сглаживанию ландшафта Batch Normalization справляется с затуханием и взрывом градиента. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnorm_helps_find_minimum_of_function.jpg" width="900">

### Советы по использованию Batch Normalization

Стоит помнить, что с батч-нормализацией:

* **Крайне важно** перемешивать объекты (составлять новые батчи) между эпохами. Единицей обучения параметров $\beta$ и $\gamma$ являются батчи. Если их не перемешивать, то из 6400 объектов в тренировочном датасете получим лишь 100 объектов (при условии, что в батче 64 объекта) для обучения $\beta$ и $\gamma$

* В слое, после которого поставили Batch Normalization, надо убрать bias (параметр $\beta$ в BatchNormalization берет эту роль на себя)


* Другое расписание learning rate: бОльшее значение в начале обучения и быстрое уменьшение в процессе обучения


* Чем меньше размер батча в обучении, тем хуже будет работать BatchNormalization

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnorm_batch_size.png" width="550">

<center><p><em>Source: <a href="https://arxiv.org/pdf/1803.08494.pdf">Group Normalization</a></p> </em></center>

### Используем Batch Normalization в PyTorch

In [ ]:
class SimpleMNIST_NN_Init_Batchnorm(nn.Module):
    def __init__(self, n_layers):
        super().__init__()
        self.n_layers = n_layers
        layers = [
            nn.Linear(28 * 28, 100, bias=False),
            nn.BatchNorm1d(100),
            nn.Sigmoid(),
        ]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100, bias=False))
            layers.append(nn.BatchNorm1d(100))
            layers.append(nn.Sigmoid())
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        return x

    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                torch.nn.init.xavier_normal_(child.weight, gain=sigmoid_gain)
                if child.bias is not None:
                    torch.nn.init.zeros_(child.bias)

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)

hooks_data_history = register_model_hooks(model)
history = train_model(model, optimizer, model_name="batchnorm2")
plot_history(history)

In [ ]:
plot_hooks_data(hooks_data_history)

Попробуем, согласно советам, увеличить learning rate:

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)

hooks_data_history = register_model_hooks(model)
history = train_model(model, optimizer, model_name="batchnorm_increased_lr")
plot_history(history)

In [ ]:
plot_hooks_data(hooks_data_history)

### Ставить Batch Normalization до или после активации?



#### До


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization_before_activation.png" width="400">

* Рекомендуется авторами статьи, где предложили Batch Normalization
* Для сигмоиды BN, поставленная после активации, не решает проблем сигмоиды

#### После

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization_after_activation.png" width="400">

* Аргументация авторов статьи не до конца обоснована
* Обычно, сигмоиду не используют в современных нейронных сетях
* Для популярной ReLU BN, поставленная до активации, может приводить к “умирающей ReLU”, когда большая часть ее входов меньше 0, и потому для них градиент не проходит
* На многих задачах BN после функции активации работает лучше или не хуже поставленной до

**BN — before or after ReLU?**

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnormalization_before_or_after_relu.png" width="500">

<center><p><em>Source: <a href="https://github.com/ducha-aiki/caffenet-benchmark/blob/master/batchnorm.md">BN experiments</a></p> </em></center>

## Другие Normalization

Существует большое количество иных нормализаций, их неполный список можно найти [здесь](https://paperswithcode.com/methods/category/normalization). В данной секции мы рассмотрим самые популярные виды нормализации помимо BatchNorm.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/normalization_methods.png" width="900">

<center><p><em>Source: <a href="https://paperswithcode.com/method/layer-normalization">Layer Normalization</a></p> </em></center>

Чаще всего нормализации тестируют на изображениях, в том числе или только на изображениях. Потому их графически объясняют при помощи следующей схемы:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/dimensions_channels_batch_samples.png" width="450">

*По оси абсцисс* расположены объекты из батча,  
*по оси ординат* &mdash; feature map, преобразованный в вектор,  
*по оси аппликат* &mdash; каналы (feature maps).

В этом представлении BatchNorm выглядит следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/visualization_of_batch_normalization.png" width="450">

[Batch Normalization](https://paperswithcode.com/method/batch-normalization)

### Layer Norm

Помимо свёрточных нейронных сетей, существует специальный тип нейронных сетей, используемых для обработки последовательностей. Называется он "рекуррентные нейронные сети", ему же будет посвящена наша следующая лекция. 

Когда оказалось, что BatchNorm положительно сказывается на обучении нейронных сетей, его попытались применить для различных архитектур. BatchNorm нельзя было использовать "из коробки" для рекуррентных нейронных сетей (работающих с последовательными данными), пришлось придумывать различные адаптации, среди которых наиболее удачной оказалась **Layer Normalization**.  
По сути, теперь нормализация происходит внутри одного объекта из батча, а не поканально в рамках батча. С математической точки зрения, данная "адаптация" отличается от BatchNorm, однако экспериментально она превзошла своих конкурентов в задаче нормализации при обработке последовательных данных.

Впоследствии данный метод нормализации хорошо проявил себя в трансформерах &mdash; наследниках рекуррентных нейронных сетей в вопросах обработки последовательных данных (об этом типе нейросетей мы также поговорим на следующей лекции). После успешного применения трансформеров в задачах компьютерного зрения, LayerNorm стал использоваться и в компьютерном зрении (хотя и уступал BatchNorm при использовании в свёрточных нейронных сетях).

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/visualization_of_layer_normalization.png" width="450">

[Layer Normalization](https://paperswithcode.com/method/layer-normalization)


### Instance Norm

Следующий вид нормализации был предложен отечественными исследователями (из Сколтеха), занимавшимися разработкой быстрого и эффективного способа переноса стиля с одного изображения на другое. 

При использовании BatchNorm терялась информация о контрастах на конкретном изображении, поскольку нормализация производится по нескольким объектам. Для сохранения *контрастов* в экземпляре (*instance*) изображения была предложена специальная нормализация, рассматривающая конкретный канал одного конкретного объекта. Было предложено два названия нормализации: связанное с мотивацией (contrast normalization) и связанное с принципом работы (**instance normalization**). 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/visualization_of_instance_normalization.png" width="450">

[Instance Normalization](https://paperswithcode.com/method/instance-normalization)

### Group Norm

В течение долгого времени BatchNorm оставался однозначным фаворитом для использования в задачах компьютерного зрения, однако:
1. В связи с необходимостью точно считать статистики внутри batch, при обучении приходилось использовать большой batch size.  

2. Ограниченность размера памяти видеокарты вынуждает разработчиков идти на компромисс между сложностью модели и batch size.

Таким образом, использование BatchNorm приводило к невозможности использовать сложные модели**\***, поскольку им просто не хватало места на видеокарте. 

Необходимость использовать большой batch size могут решать различные нормализации, не использующие batch-размерность. К примеру, уже известные нам **Layer Norm** и **Instance Norm**. Эмпирически оказалось, что данные нормализации уступают BatchNorm по качеству работы: в то время как LayerNorm предполагает одинаковую важность и суть различных каналов (*рассматривая данные излишне глобально*), InstNorm упускает межканальные взаимодействия (*рассматривая данные слишком локально*). 

Успешным обобщением данных методов является **Group Normalization**: данный метод разбивает каналы на $G \in [1; C]$ групп, присваивая каждой из них (примерно) равное количество каналов. Отметим, что при $G = 1$, GroupNorm идентичен LayerNorm, а при $G = C$, GroupNorm идентичен InstNorm. 

Эмпирически оказалось, что при замене BatchNorm на GroupNorm качество модели падает в разы менее значительно, чем при использовании LayerNorm либо GroupNorm. Более того, при изменении batch size качество работы LayerNorm не изменялось, что открыло перспективы для создания более сложных моделей компьютерного зрения. 

**\*** — подразумевается, что уменьшение batch size позволило бы создать более сложные модели.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/visualization_of_group_normalization.png" width="450">

[Group Normalization](https://paperswithcode.com/method/group-normalization)

# Регуляризация

Второй способ улучшения сходимости нейросетей и борьбы с переобучением — введение регуляризации. Ее можно вводить несколькими способами. 

## L1, L2 регуляризации

Мы уже разбирали самый простой способ &mdash; добавление штрафа к весам в функцию потерь. На сходимость нейросети, это, правда, влияет слабо.

$$Loss\_reg = loss + \lambda \cdot reg$$

$$ reg_{L1} = \lambda \sum |w_i| $$

$$ reg_{L2} = \lambda \sum w_i^2 $$

<img src="https://edunet.kea.su/repo/EduNet-content/L07/out/l1_l2_regularization_to_weight.gif" alt="alttext" width="500px"/>

[Visualizing regularization and the L1 and L2 norms](https://towardsdatascience.com/visualizing-regularization-and-the-l1-and-l2-norms-d962aa769932)

Иногда уже его хватает, чтобы решить все проблемы. Напомним, что L2 лосс приводит к большому числу маленьких ненулевых весов в сети. А L1 лосс — к маленькому числу ненулевых весов (разреженной нейросети)

## Dropout

Одним из распространенных именно в нейросетях методом регуляризации является Dropout.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/dropout.png" width="700">

Состоит этот метод в следующем:

1. Во время обучения мы с вероятностью $p$ зануляем выход нейронов слоя (например, $p$ = 0.5)
2. Зануленные нейроны не участвуют в данном `forward`, и поэтому градиент к ним при `backward` не идет. 

3. Сила регуляризации определяется вероятностью $p$: чем она больше, тем сильнее регуляризация. 


### Мотивация Dropout

### Борьба с коадаптацией 

Одной из  проблем при работе с глубокими сетями является совместная адаптация нейронов. В такой сети, если все весовые коэффициенты изучаются вместе, обычно некоторые соединения будут иметь больше возможностей прогнозирования, чем другие.

Часть нейронов делает основную работу &mdash; предсказывает, а остальные могут вообще не вносить никакого вклада в итоговое предсказание. Или же другая картина: один нейрон делает неверное предсказание, другие его исправляют, и в итоге первый нейрон свои ошибки не исправляет. 

Это явление называется коадаптацией. Этого нельзя было предотвратить с помощью традиционной регуляризации, такой как L1 и L2. А вот Dropout с этим хорошо борется

Отмена способности некоторых нейронов учиться во время тренировки на самом деле, оказывается, направлена на то, чтобы получить более обученные нейроны и сократить переобучение. Она гарантирует, что выучиваемые индивидуальными нейронами функции хорошо работают со случайно выбранными подмножествами функций, выученных другими нейронами. 
Таким образом, Dropout приводит к появлению более требовательных нейронов, которые пытаются обойти сложные, индивидуальные особенности, которые склонны плохо обобщать, и сохраняют больше полезной информации самостоятельно. 

На следующем рисунке, извлеченном из [статьи](https://jmlr.org/papers/v15/srivastava14a.html), мы находим сравнение признаков, изученных в наборе данных MNIST с одним скрытым слоем в автоэнкодере, имеющим 256 признаков после ReLU без dropout (слева), и признаков, изученных той же структурой с использованием dropout в ее скрытом слое с $p$ = 0.5 (справа).

В то время как первый показывает неструктурированные, беспорядочные паттерны, которые невозможно интерпретировать, второй явно демонстрирует целенаправленное распределение веса, которое обнаруживает штрихи, края и пятна самостоятельно, нарушая их взаимозависимость с другими нейронами для выполнения этой работы.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/compare_weights_with_dropout_and_without_dropout.png" width="600">

<center><p><em>Source: <a href="https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf">Dropout: A Simple Way to Prevent Neural Networks from
Overfitting
</a></p> </em></center>

### Dropout как регуляризация 

Фактически, Dropout штрафует слишком сложные, неустойчивые решения. Добавляя в нейросеть Dropout, мы сообщаем ей о том, что решение, которое мы ожидаем, должно быть устойчиво к шуму.

### Dropout как ансамбль 

Можно рассматривать Dropout как ансамбль нейросетей со схожими параметрами, которые мы учим одновременно, вместо того, чтобы учить каждую в отдельности, а затем результат их предсказания усредняем, [замораживая Dropout](https://prvnk10.medium.com/ensemble-methods-and-the-dropout-technique-95f36e4ae9be).

Фактически, возникает аналогия со случайным лесом: каждая из наших нейросетей легко выучивает выборку и переобучается &mdash; имеет низкий bias, но высокий variance. При этом за счет временного отключения активаций, каждая нейросеть видит не все объекты, а только часть. Усредняя все эти предсказания, мы уменьшаем variance. 



### Dropout помогает бороться с переобучением

Dropout, в силу указанного выше, может хорошо помогать бороться с переобучением.

И в случае линейных слоев

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/dropout_solve_overfitting_problem_in_mlp_networks.png" width="500">

<center><p><em>Source: <a href="https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/">Tutorial: Dropout as Regularization and Bayesian Approximation</a></p> </em></center>

И в случае свёрточных слоёв 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/dropout_solve_overfitting_problem_in_convolution_networks.png" width="500">

<center><p><em>Source: <a href="https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/">Tutorial: Dropout as Regularization and Bayesian Approximation</a></p> </em></center>

### Простая реализация Dropout

Напишем "наивную" реализацию модуля Dropout.

In [ ]:
class BadDropout(nn.Module):
    def __init__(self, p: float = 0.5):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError(
                f"Dropout probability has to be between 0 and 1, but got {p}"
            )
        self.p = p

    def forward(self, x):
        if self.training:
            keep = torch.rand(x.size()) > self.p
            if x.is_cuda:
                keep = keep.to(device)
            return x * keep
        # in test time, expectation is calculated
        return x * (1 - self.p)

Приведенная реализация неоптимальна, так как в режиме инференса (когда ```training = False```), функция ```forward``` совершает дополнительное умножение. Одним из приоритетов при создании модели является скорость работы в режиме инференса. Поэтому по возможности все "лишние" операции выполняют только в режиме обучения. В данном случае можно целиком убрать коэффициент нормировки из режима инференса, перенеся его в режим обучения в знаменатель.

Дополнительным плюсом такого подхода является то, что при удалении модуля из архитектуры сети функция прогнозирования не изменится.

In [ ]:
class Dropout(nn.Module):
    def __init__(self, p: float = 0.5):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError(
                f"Dropout probability has to be between 0 and 1, but got {p}"
            )
        self.p = p

    def forward(self, x):
        if self.training:
            keep = torch.rand(x.size()) > self.p
            if x.is_cuda:
                keep = keep.to(device)
            return x * keep / (1 - self.p)
        return x  # in test time, expectation is calculated intrinsically - we just not divide weights

Попробуем применить Dropout в нашей нейросети:

In [ ]:
class SimpleMNIST_NN_Dropout(nn.Module):
    def __init__(self, n_layers, activation=nn.Sigmoid, init_form="normal"):
        super().__init__()
        self.n_layers = n_layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100))
            layers.append(Dropout())  # add Dropout
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init_form = init_form
        if self.init_form is not None:
            self.init()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        return x

    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                if self.init_form == "normal":
                    torch.nn.init.xavier_normal_(child.weight, gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                elif self.init_form == "uniform":
                    torch.nn.init.xavier_uniform_(child.weight, gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                else:
                    raise NotImplementedError()

Так как наша модель из-за Dropout ведет себя по-разному во время обучения и во время тестирования, то мы должны прямо ей сообщать, обучается она сейчас или нет. Делается это при помощи функций `model.train` и `model.eval`

In [ ]:
from tqdm import tqdm


def train_model_sep(model, optimizer, model_name=None, num_epochs=5):

    criterion = nn.CrossEntropyLoss().to(device)

    train_history = {}
    train_history["model_name"] = model_name
    train_history["loss_on_train"] = []
    train_history["loss_on_test"] = []

    for epoch in tqdm(range(num_epochs)):
        model.train()
        loss_on_train = train_epoch(model, optimizer, criterion, train_loader)
        model.eval()
        _, loss_on_test = validate(model, criterion, test_loader)
        train_history["loss_on_train"].extend(loss_on_train)
        train_history["loss_on_test"].extend(loss_on_test)
    return train_history

Обучим модель с Dropout:

In [ ]:
model = SimpleMNIST_NN_Dropout(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

history = train_model_sep(model, optimizer, model_name="nn3_dropout")
plot_history(history)

В данном случае выигрыша мы не получили. Возможно, если учить нейросеть больше эпох, эффект бы заметили

### Пример борьбы с переобучением при помощи Dropout
 

Чтобы увидеть эффект и при этом не учить нейросеть 100+ эпох, сделаем искусственный пример.

Просто добавим к линейной зависимости шум и попробуем выучить ее нейронной сетью.

[Dropout in Neural Networks with PyTorch](https://towardsdatascience.com/batch-normalization-and-dropout-in-neural-networks-explained-with-pytorch-47d7a8459bcd)

In [ ]:
N = 50  # number of data points
noise = 0.3

# generate the train data
x_train = torch.unsqueeze(torch.linspace(-1, 1, N), 1)
y_train = x_train + noise * torch.normal(torch.zeros(N, 1), torch.ones(N, 1))

# generate the test data
x_test = torch.unsqueeze(torch.linspace(-1, 1, N), 1)
y_test = x_test + noise * torch.normal(torch.zeros(N, 1), torch.ones(N, 1))

print(f"x_train shape: {x_train.shape}\nx_test shape: {x_test.shape}")

In [ ]:
plt.scatter(
    x_train.data.numpy(), y_train.data.numpy(), c="purple", alpha=0.5, label="train"
)
plt.scatter(
    x_test.data.numpy(), y_test.data.numpy(), c="yellow", alpha=0.5, label="test"
)

x_real = np.arange(-1, 1, 0.01)
y_real = x_real
plt.plot(x_real, y_real, c="green", label="true")
plt.legend()
plt.show()

Модель без Dropout:

In [ ]:
N_h = 100  # num of neurons
model = torch.nn.Sequential(
    torch.nn.Linear(1, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Модель с Dropout:

In [ ]:
N_h = 100  # num of neurons

model_dropout = nn.Sequential(
    nn.Linear(1, N_h),
    nn.Dropout(0.5),  # 50 % probability
    nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    nn.Dropout(0.2),  # 20% probability
    nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
optimizer_dropout = torch.optim.Adam(model_dropout.parameters(), lr=0.01)

In [ ]:
num_epochs = 1500
criterion = torch.nn.MSELoss()

for epoch in range(num_epochs):

    # train without dropout
    y_pred = model(x_train)  # look at the entire data in a single shot
    loss = criterion(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # train with dropout
    y_pred_dropout = model_dropout(x_train)
    loss_dropout = criterion(y_pred_dropout, y_train)
    optimizer_dropout.zero_grad()
    loss_dropout.backward()
    optimizer_dropout.step()

    if epoch % 100 == 0:

        model.eval()  # not train mode
        model_dropout.eval()  #  not train mode

        # get predictions
        y_test_pred = model(x_test)
        test_loss = criterion(y_test_pred, y_test)

        y_test_pred_dropout = model_dropout(x_test)
        test_loss_dropout = criterion(y_test_pred_dropout, y_test)
        # plotting data and predictions
        plt.scatter(
            x_train.data.numpy(),
            y_train.data.numpy(),
            c="purple",
            alpha=0.5,
            label="train",
        )
        plt.scatter(
            x_test.data.numpy(),
            y_test.data.numpy(),
            c="yellow",
            alpha=0.5,
            label="test",
        )
        plt.plot(
            x_test.data.numpy(), y_test_pred.data.numpy(), "r-", lw=3, label="normal"
        )
        plt.plot(
            x_test.data.numpy(),
            y_test_pred_dropout.data.numpy(),
            "b--",
            lw=3,
            label="dropout",
        )

        plt.title(
            "Epoch %d, Loss = %0.4f, Loss with dropout = %0.4f"
            % (epoch, test_loss, test_loss_dropout)
        )

        plt.legend()

        model.train()  # train mode
        model_dropout.train()  # train mode

        plt.pause(0.05)

Видим, что нейросеть без Dropout сильно переобучилась.

### Confidence interval от Dropout

Можно, используя нейросеть с Dropout, получить доверительный интервал для нашего предсказания (как делали в лекции по ML). Просто не "замораживаем" dropout-слои во время предсказания, а делаем предсказания с активными dropout. 

И делаем forward через такую нейросеть для одного объекта 1000 раз. 
Сделав это 1000 раз, вы получаете распределение предсказаний, на основе которого можно делать confidence интервалы и как раз ловить те объекты, на которых нейросеть вообще не понимает, что ей делать, и потому предсказывает метку или еще что-то с сильной дисперсией. 


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/confidence_interval_dropout.png" width="600">



## DropConnect

Если занулять не нейроны (активации), а случайные веса, с вероятностью $p$,получится DropConnect.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/dropconnect.png" width="650">

DropConnect похож на Dropout, поскольку он вводит динамическую разреженность в модель, но отличается тем, что разреженность зависит от весов *W*, а не от выходных векторов слоя. Другими словами, полностью связанный слой с DropConnect становится разреженно связанным слоем, в котором соединения выбираются случайным образом на этапе обучения. 

В принципе, вариантов зануления чего-то в нейронной сети можно предложить великое множество, в разных ситуациях будут работать разные способы ([в этом списке](https://paperswithcode.com/methods/category/regularization)  много Drop...).

## DropBlock

Например, можно убирать для каждого батча из нейросети случайные блоки из слоев. И это будет работать!


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/dropblock.png" width="750">

[Why Self-training with Noisy Students beats SOTA Image classification while using fewer resources](https://medium.datadriveninvestor.com/the-next-big-thing-in-image-classification-self-training-with-noisy-student-for-improving-22d52dc74dda)

## Batch Normalization до или после Dropout



### До

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization_before_dropout.png" width="600">

* Меньше влияние (covariate shift) Dropout на BatchNorm

### После

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/batchnormalization_after_dropout.png" width="600">

* Информация о зануленных активациях не просачивается через среднее и дисперсию батча

### Ставить только что-то одно 

* Dropout может отрицательно влиять на качество нейросети с BatchNorm за счет разного поведения на train и test

### Строго говоря

* Оптимальный порядок следования слоев зависит от задачи и архитектуры сети
* Возможно, стоит применять модифицированные версии BatchNorm
* Если используем BatchNormalization, то надо уменьшить силу Dropout и L2-регуляризации

# Оптимизация параметров нейросетей

В процессе обучения мы пытаемся подобрать параметры модели при которых она будет работать лучше всего. Это — **оптимизационная задача** (задача подбора оптимальных параметров). Мы уже ознакомились с одним алгоритмом оптимизации параметров — **градиентным спуском**. 

Существует множество **алгоритмов оптимизации**, которые можно применять для поиска минимума функционала ошибки ([неполный список](https://paperswithcode.com/methods/category/stochastic-optimization)). Эти алгоритмы реализованы в модуле [`torch.optim`](https://pytorch.org/docs/stable/optim.html).

Важно отметить, что **выбор оптимизатора не влияет на расчет градиента**. Градиент в PyTorch вычисляется автоматически на основе графа вычисления. 


## Обзор популярных оптимизаторов

### SGD (stochastic gradient descent)

При градиентном спуске мы:
- делаем прямой проход, вычисляем функционал ошибки $L(x, y, w_t)$
- делаем обратный проход, вычисляем градиент $\nabla_wL(x, y, w_t)$
- делаем шаг оптимизации: изменяем параметры модели по формуле:

$$w_{t+1} = w_t - lr \cdot \nabla_wL(x, y, w_t),$$

домножая антиградиент на постоянный коэффициент $lr$ (гиперпараметр обучения &mdash; learning rate).

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/stochastic_gradient_descent.gif" width="950">

У данного алгоритма есть проблема: он может застревать в **локальных минимумах** или даже **седловых точках**. 

**Cедловые точки** — точки, в которых все производные равны 0, но они не являются экстремумами. В них градиент равен 0, веса не обновляются — оптимизация останавливается.

Пример таких точек:

- Точка 0 у функции $x^3$, не имеющей минимума или максимума вовсе 
- Точка (0, 0) у функции $z = x^2 - y^2$

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/getting_stuck_in_local_minimum_example.png" width="350"> <img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/saddle_point_example.png" width="400">

[Седловая точка
](https://ru.wikipedia.org/wiki/%D0%A1%D0%B5%D0%B4%D0%BB%D0%BE%D0%B2%D0%B0%D1%8F_%D1%82%D0%BE%D1%87%D0%BA%D0%B0)

Частично эту проблему решает **стохастический градиентный спуск** (stochastic gradient descent, **SGD**). В нем для градиентного спуска используется не все данные, а некоторая подвыборка (mini-batch), или даже один элемент. 


**SGD** обладает важной особенностью: на каждом объекте или подвыборке (mini-batch) ландшафт функции потерь выглядит по-разному. Некоторые минимумы функции потерь и седловые точки могут быть характерны лишь для части объектов.

Ниже **упрощенно** показаны ландшафты функции потерь на полном датасете и на отдельных батчах. При оптимизации на полном датасете модель могла бы **остановиться в левом локальном минимуме**, но стохастическая природа спуска позволяет избежать этого за счет того, что **для некоторых батчей этот минимум отсутствует**. 

В результате модель сможет остановиться только в каком-то относительно широком и глубоком минимуме, характерном для большинства батчей обучающих данных. С большой вероятностью этот минимум будет присутствовать и на реальных данных, то есть модель сможет адекватно работать с ними.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/sgd_loss_batch_landscape.png" width="700">

**SGD** до сих пор является достаточно популярным методом обучения нейросетей, потому что он простой, не требует подбора дополнительных гиперпараметров, кроме **скорости обучения** `lr`, и сам по себе обычно дает неплохие результаты. 

Если же модель учится слишком долго и/или важна каждая сотая в качестве, то нужно либо использовать его в совокупности с другими техниками (их рассмотрим далее), либо использовать другие способы.

Фрагмент кода для понимания работы SGD:
```
class SGD:  
  def __init__(self, parameters, lr):
    self.parameters = parameters
    self.lr = lr
  
  def step(self):
    d_parameters = self.parameters.grad
    self.parameters -= self.lr * d_parameters
```

Алгоритм SGD реализован в [`torch.optim.SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [ ]:
import torch.optim as optim

parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], lr=0.001)

**Минусы SGD**:

 1. Если функция ошибки быстро меняется в одном направлении, а в другом &mdash; медленно, то это приводит к резким изменениям направления градиентов и замедляет процесс обучения.


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/stohastic_gradient_descent_no_momentum.gif" width="500">


[Machine Learning Optimization Methods “Mechanics, Pros and Cons”](https://salmenzouari.medium.com/machine-learning-optimization-methods-mechanics-pros-and-cons-81b720194292)

 2. Может застревать в локальных минимумах или седловых точках. 

 3. Так как мы оцениваем градиент по малой части выборки, они могут плохо отображать градиент по всей выборке и являться шумными. В результате часть шагов градиентного спуска делаются впустую или во вред. 
 
 4. Мы применяем один и тот же `learning rate` ко всем параметрам, что не всегда разумно. Параметр, отвечающий редкому классу, будет обучаться медленнее остальных. 
 
 5. Просто медленнее сходится.

Основой всех описанных ниже алгоритмов является SGD.

### Momentum

Чтобы избежать проблем 1—3, можно добавить движению по ландшафту функции ошибок инерции (**momentum**). По аналогии с реальной жизнью: если мяч катится с горки, то он, благодаря инерции, может проскочить пологое место или даже небольшую яму.  

Корректируем направление движения шарика с учетом текущего градиента:

$$v_{t} = m \cdot v_{t-1} + \nabla_wL(x, y, w_{t})$$

где $m \in [0, 1)$ — momentum гиперпараметр.

Вычисляем, куда он покатится:

$$w_{t+1} = w_t - lr \cdot v_{t}$$

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/advantages_wtih_momentum.png" width="480">

[Градиентный спуск, как учатся нейронные сети (видео)](https://youtu.be/IHZwWFHWa-w)

Теперь мы быстрее достигаем локального минимума и можем выкатываться из совсем неглубоких. Градиент стал менее подвержен шуму, меньше осциллирует

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/stohastic_gradient_descent_no_momentum.gif" width="500"> <img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/stohastic_gradient_descent_with_momentum.gif" width="500">

Фрагмент кода для понимания работы Momentum:

```
class SGD_with_momentum:  
  def __init__(self, parameters, momentum, lr):
    self.parameters = parameters
    self.momentum = momentum
    self.lr = lr
    self.velocity = torch.zeros_like(parameters)
    
  def step(self):
    d_parameters = self.parameters.grad
    self.velocity =  self.momentum * self.velocity + d_parameters
    self.weights -= self.lr * self.velocity
```


Алгоритм Momentum реализован в [`torch.optim.SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [ ]:
import torch.optim as optim

parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], momentum=0.9, lr=0.001)

У этого подхода есть одна опасность — мы можем выкатиться за пределы минимума, к которому стремимся, а потом какое-то время к нему возвращаться. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/problem_of_big_momentum_value.gif" width="700">

<center><p><em>Source: <a href="https://distill.pub/2017/momentum/">Why Momentum Really Works</a></p> </em></center>

[optimizer-visualization](https://github.com/Jaewan-Yun/optimizer-visualization)

Чтобы с этим бороться, предложен другой способ подсчета инерции

### NAG (Nesterov momentum)

Будем сначала смещаться в сторону, куда привел бы нас наш накопленный градиент, там считать новый градиент и смещаться по нему. 
В результате перескоки через минимум будут менее значительными, и мы будем быстрее сходиться

$$v_{t} = m \cdot v_{t-1} +  \nabla_w L(w_t - lr \cdot m \cdot  v_{t-1} )$$

$$w_{t+1} = w_{t} - lr \cdot v_{t} $$

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/nesterov_momentum.png" width="800">

Кажется, что для реализации такого алгоритма необходимо пересчитывать прямой и обратный проход с новыми параметрами, для вычисления градиента. На практике эту формулу можно [переписать](http://www.cs.toronto.edu/~hinton/absps/momentum.pdf) так, чтобы не пересчитывать градиент. 

С псевдокодом, описывающим последовательность действий NAG можно познакомиться в [документации PyTorch](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html).

Алгоритм Nesterov momentum реализован в [`torch.optim.SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [ ]:
import torch.optim as optim

parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], momentum=0.9, nesterov=True, lr=0.001)

### Adaptive Learning Rate

Описанные алгоритмы не борются с 4-ой проблемой SGD: "мы применяем **один и тот же learning rate ко всем параметрам**, что не всегда разумно. Параметр, отвечающий редкому классу, будет обучаться медленнее остальных". 

**Пример:** мы решаем задачу классификации картинок из Интернета и у нас есть параметры, ответственные за признаки, которые характеризуют кошек породы сфинкс. Кошки породы сфинкс встречаются в нашем датасете редко и эти параметры реже получают информацию для обновления. Поэтому наша модель может хуже классифицировать кошек этой породы.

Для решения этой проблемы мы можем завести для каждого параметра **индивидуальный learning rate**, зависящий от того, как часто и как сильно изменяется этот параметр в процессе обучения.

### Adagrad

Будем хранить для каждого параметра **сумму квадратов его градиентов** (запоминаем как часто и как сильно он изменялся).

И будем вычитать из значений параметров градиент с коэффициентом, обратно пропорциональным корню из этой суммы $G_t$. 

$$ G_ t = \sum_{i=1}^t \nabla_w L(x,y,w_i)\odot\nabla_w L(x,y,w_i) $$

$$ w_{t+1} = w_{t} -  \frac{lr}{\sqrt{G_t} + e} \odot \nabla_w L(x,y,w_{t}) $$

$e$ — малая константа, чтобы не допускать деления на ноль, $\odot$ — поэлементное умножение.

В результате, если градиент у нашего веса часто большой, коэффициент будет уменьшаться. 

Проблема заключается в том, что при такой формуле наш `learning rate` неминуемо в конце концов затухает (так как сумма квадратов не убывает).


Фрагмент кода для понимания работы Adagrad:
```
class AdaGrad:  
  def __init__(self, parameters, lr=0.01):
     self.parameters = parameters
     self.lr = lr
     self.grad_squared = torch.zeros_like(parameters)
   
  def step(self):
    d_parameters = self.parameters.grad
    self.grad_squared += d_parameters * d_parameters
    self.parameters -= self.lr * d_parameters / (torch.sqrt(self.grad_squared) + 1e-7)
```



Алгоритм Adagrad реализован в [`torch.optim.Adagrad`](https://pytorch.org/docs/stable/generated/torch.optim.Adagrad.html)

In [ ]:
import torch.optim as optim

parameters = torch.randn(10, requires_grad=True)
optimizer = optim.Adagrad([parameters], lr=0.01)

### RMSprop

Добавим "забывание" предыдущих квадратов градиентов. Теперь мы считаем не сумму квадратов, а [экспоненциальное скользящее среднее](https://ru.wikipedia.org/wiki/%D0%A1%D0%BA%D0%BE%D0%BB%D1%8C%D0%B7%D1%8F%D1%89%D0%B0%D1%8F_%D1%81%D1%80%D0%B5%D0%B4%D0%BD%D1%8F%D1%8F) с коэффициентом $\alpha$. 


$$G_t = \alpha \cdot G_{t-1} + (1-\alpha) \cdot \nabla_w L(x,y,w_t) \odot \nabla_w L(x,y,w_t)$$

$$w_{t+1} = w_{t} - \frac{lr}{\sqrt{G_t }+ e} \odot \nabla_w L(x,y,w_t)$$

Фрагмент кода для понимания работы RMSprop:

```
class RMSprop():  
  def __init__(self, parameters, lr=0.01, alpha=0.99):
    self.parameters = parameters
    self.lr = lr
    self.alpha = alpha
    self.grad_squared = torch.zeros_like(parameters)
  
  def step(self):
    d_parameters = self.parameters.grad
    self.grad_squared = self.alpha * self.grad_squared +\
        (1 - self.alpha) * d_parameters * d_parameters

    self.parameters -= self.lr * d_parameters / (torch.sqrt(self.grad_squared) + 1e-7)
```



Алгоритм RMSprop реализован в [`torch.optim.RMSprop`](https://pytorch.org/docs/stable/generated/torch.optim.RMSprop.html).

In [ ]:
import torch.optim as optim

parameters = torch.randn(10, requires_grad=True)
optimizer = optim.RMSprop([parameters], lr=0.01, alpha=0.99)

### Adam

Одним из самых популярных адаптивных оптимизаторов является Adam, объединяющий идеи momentum и adaptive learning rate:

$$ v_t = \beta_1 \cdot v_{t-1} + (1-\beta_1) \cdot \nabla_w L(x,y,w_t) $$

$$ G_t = \beta_2 \cdot G_{t-1} + (1-\beta_2) \cdot \nabla_w L(x,y,w_t) \odot \nabla_w L(x,y,w_t) $$

$$ w_{t+1} = w_t - \frac{lr}{\sqrt{G_t} + e} \odot v_t$$

где $\beta_1$ — аналог $m$ из Momentum, а $\beta_2$ — аналог $\alpha$ из RMSprop.

Фрагмент кода для понимания работы Adam:

```
class Adam:  
  def __init__(self, parameters, lr=0.01, betas=(0.9, 0.999)):
    self.parameters = parameters
    self.lr = lr
    self.betas = betas
    self.velocity = torch.zeros_like(parameters) 
    self.grad_squared = torch.zeros_like(parameters)   
    self.beta_1 = betas[0] # momentum
    self.beta_2 = betas[1] # alpha
  
  def step(self):
    d_parameters = self.parameters.grad
    # momentum
    self.velocity = self.beta_1  *self.velocity + (1 - self.beta_1) * d_parameters
    # adaptive learning rate
    self.grad_squared = self.beta_2 * self.grad_squared + \
        (1 - self.beta_2) * d_parameters * d_parameters
    self.parameters -= self.lr * self.velocity / (torch.sqrt(self.grad_squared) + 1e-7)
```



Чтобы в начале у нас получались очень большие шаги, будем дополнительно модицифировать инерцию и сумму квадратов

$$ v_t = \frac{v_t}{1-\beta_1^t} $$

$$ G_t = \frac{G_t}{1-\beta_2^t} $$

Алгоритм Adam реализован в [`torch.optim.Adam`](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html).

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.Adam([parameters], betas=(0.9, 0.999))

Пример применения

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
hooks_data_history = register_model_hooks(model)

history = train_model_sep(model, optimizer, model_name="adam")
plot_history(history)

In [ ]:
plot_hooks_data(hooks_data_history)

### L2 vs Weight decay

Для использования L2 c оптимизатором необходимо указать значение `weight_decay`,  где `weight_decay` — коэффициент перед L2.

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.RMSprop([parameters], alpha=0.99, weight_decay=0.001)

Вообще говоря, Weight decay и L2 — это немного разные вещи. 
L2 добавляет член регуляризации к Loss функции:

$$Loss_{L2} = Loss + \frac{λ}{2n}w^2$$

Weight decay уменьшает веса:

$$w_{wd} = w - \frac{λ}{n}w$$

где $λ$ — константа, а $n$ — количество элементов в батче. 


Для **SGD** оптимизатора Weight decay и L2 — **эквивалентны**, но не для всех оптимизаторов это так. 

Например, это не так для Adam (подробно об этом можно почитать [тут](https://arxiv.org/pdf/1711.05101.pdf)). 

Обратите внимание, что `weight_decay` в `torch.optim.Adam` — это коэффициент перед L2. Weight decay для Adam реализовано в 
`torch.optim.AdamW`. 
[Считается](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd), что Weight decay для Adam работает лучше чем L2, но на практике это не всегда выполняется. 


## Сравнение оптимизаторов 

У каждого из предложенных оптимизаторов есть минусы и плюсы 

- Методы с инерцией сходятся к решению более плавно, но могут "перелетать"

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/convergence_optimizers.gif" width="250">

<center><p><em>Source: <a href="https://imgur.com/a/Hqolp">Visualizing Optimization Algos</a></p> </em></center>



* Методы с адаптивным learning rate быстрее сходятся, более стабильны и меньше случайно блуждают

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/methods_with_adaptive_learning_rate.gif" width="250">

<center><p><em>Source: <a href="https://imgur.com/a/Hqolp">Visualizing Optimization Algos</a></p> </em></center>


* Алгоритмы без адаптивного learning rate сложнее выбираются из локальных минимумом

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/methods_without_adaptive_learning_rate.gif" width="450">

<center><p><em>Source: <a href="https://imgur.com/a/Hqolp">Visualizing Optimization Algos</a></p> </em></center>

* Алгоритмы с инерцией осцилируют в седловых точках прежде чем найти верный путь

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/methods_with_momentum_in_saddle_point.gif" width="450">

<center><p><em>Source: <a href="https://imgur.com/a/Hqolp">Visualizing Optimization Algos</a></p> </em></center>


# Режимы обучения

Нам не обязательно поддерживать один и тот же `learning rate` в течение всего обучения. Более того, для того же SGD есть гарантии, что если правильно подобрать схему уменьшения `learning rate`, он сойдется к глобальному оптимуму.

Мы можем менять `learning rate` по некоторым правилам.  

## Ранняя остановка

Можем использовать критерий ранней остановки: когда значение функции потерь на валидационной выборке не улучшается какое-то количество эпох(`patience`), умножаем `learning rate` на некое значение `factor`).

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/early_stopping.png" width="500">

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, "min", factor=0.1, patience=5
)

Применим к нашей модели

(выполнение занимает ~ 5 минут)

In [ ]:
def train_model_sep_scheduler(
    model, optimizer, scheduler, model_name=None, num_epochs=5
):

    criterion = nn.CrossEntropyLoss().to(device)

    train_history = {}
    train_history["model_name"] = model_name
    train_history["loss_on_train"] = []
    train_history["loss_on_test"] = []

    for epoch in tqdm(range(num_epochs)):
        model.train()
        loss_on_train = train_epoch(model, optimizer, criterion, train_loader)
        model.eval()
        val_loss, loss_on_test = validate(model, criterion, test_loader)
        train_history["loss_on_train"].extend(loss_on_train)
        train_history["loss_on_test"].extend(loss_on_test)
        scheduler.step(val_loss)
    return train_history

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, "min", factor=0.1, patience=1
)


hooks_data_history = register_model_hooks(model)
history = train_model_sep_scheduler(
    model, optimizer, scheduler, model_name="reduce_lr_on_plateu", num_epochs=15
)
plot_history(history, num_epochs=15)

## Понижение шага обучения на каждой эпохе 

Домножать `learning rate` на `gamma` каждую эпоху

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

## Cyclical learning schedule


Мы можем не все время понижать learning rate, а делать это циклически: то понижать, то повышать. Делать это можно по-разному:


1. Постоянно оставлять одни и те же границы

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/cyclical_learning_schedule_permanent_confines.png" width="600">


2. Уменьшать верхнюю границу во сколько-то раз

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/cyclical_learning_schedule_reduce_confines.png" width="600">

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/out/cyclical_learning_schedule_reduce_confines_smooth.png" width="600">

Нюанс &mdash; здесь мы ОБЯЗАТЕЛЬНО должны хранить модель с лучшим качеством. Такая оптимизация не гарантирует, что в конце модель будет лучше, чем на каком-то промежуточном шаге

### Подбираем границы learning rate



1. Просто берем и либо делаем для каждого `learning rate` минимизацию одну эпоху (чтобы всю выборку увидеть), нейросетку для каждого значения `learning rate` инициализируем заново

Это долго

Потому  делаем "магию":

2. На каждое значение `learning rate` у нас будет лишь один батч, нейросетку для каждого `learning rate` не меняем. И это часто работает, так как нам нужна просто грубая прикидка

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/optimal_learning_rate_range.png" width="650">

<center><p><em>Source: <a href="https://medium.com/modern-nlp/transfer-learning-in-nlp-f5035cc3f62f">Transfer Learning In NLP</a></p> </em></center>



Полученные значения (оптимальные границы)  используем в качестве верхнего и нижнего порогов на `learning rate`.

In [ ]:
start_lr = 1e-8
end_lr = 10
lr_find_epochs = 2
steps = lr_find_epochs * len(train_loader)
smoothing = 0.05

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
import math

lrs = []
losses = []
optimizer = optim.SGD(model.parameters(), lr=1e-8)
lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (steps))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

for epoch in tqdm(range(lr_find_epochs)):
    for batch in train_loader:
        optimizer.zero_grad()
        x_train, y_train = batch
        x_train, y_train = x_train.to(device), y_train.to(device)
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)

        loss.backward()
        optimizer.step()
        scheduler.step()

        loss = loss.detach().cpu().numpy()
        if len(losses) > 1:
            loss = smoothing * loss + (1 - smoothing) * losses[-1]
        losses.append(loss)
        lr_step = optimizer.state_dict()["param_groups"][0]["lr"]
        lrs.append(lr_step)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("LR range selection", size=15)
plt.plot(np.log10(lrs), losses)
plt.xlabel("Learning rate (log10)", size=15)
plt.ylabel("Loss")
plt.show()

На этом графике нам нужен минимум. Это в районе 1e-1. Делим это число на 10. 
Нижняя граница, стало быть, равна 1e-2.

Снижение loss начинается с `learning rate` в районе 1e-4. Его делим на 6:

In [ ]:
min_lr = 1e-4 / 6
max_lr = 1e-2

Этот `scheduler` надо применять после каждого батча. Потому перепишем `train_epoch_sh`

In [ ]:
def train_epoch_sh(model, optimizer, scheduler, criterion, train_loader):
    loss_history = []
    for batch in train_loader:
        optimizer.zero_grad()
        x_train, y_train = batch  # parse data
        x_train, y_train = x_train.to(device), y_train.to(device)  # compute on gpu
        y_pred = model(x_train)  # get predictions
        loss = criterion(y_pred, y_train)  # compute loss
        loss_history.append(loss.cpu().detach().numpy())  # write loss to log
        loss.backward()
        optimizer.step()
        scheduler.step()
    return loss_history

### Запускаем

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=min_lr)

scheduler = optim.lr_scheduler.CyclicLR(
    optimizer, base_lr=min_lr, max_lr=max_lr, mode="triangular"
)  # first case

По-хорошему, в коде ниже мы на каждой эпохе сохраняем лучшую модель. 

In [ ]:
from copy import deepcopy


def train_model_cycle_sh(model, optimizer, scheduler, model_name=None, num_epochs=5):

    criterion = nn.CrossEntropyLoss().to(device)

    train_history = {}
    train_history["model_name"] = model_name
    train_history["loss_on_train"] = []
    train_history["loss_on_test"] = []

    best_loss = np.inf

    for epoch in tqdm(range(num_epochs)):
        model.train()
        loss_on_train = train_epoch_sh(
            model, optimizer, scheduler, criterion, train_loader
        )
        model.eval()
        val_loss, loss_on_test = validate(model, criterion, test_loader)
        train_history["loss_on_train"].extend(loss_on_train)
        train_history["loss_on_test"].extend(loss_on_test)
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = deepcopy(model)
    return best_model, train_history


hooks_data_history = register_model_hooks(model)
best_model, history = train_model_cycle_sh(
    model, optimizer, scheduler, model_name="sgd_cycle_lr", num_epochs=15
)
plot_history(history, num_epochs=15)

## Neural Network WarmUp

Также, для достаточно больших нейронных сетей практикуют следующую схему (**gradual warmup**, [изначальная статья](https://arxiv.org/pdf/1706.02677.pdf)):

Поставить изначальный `learning rate` значительно ниже того, с которого мы обычно начинаем обучение. За несколько эпох, например, 5, довести `learning rate` от этого значения до требуемого. За счет этого нейросеть лучше "адаптируется" к нашим данным. 

Также такой `learning schedule` позволяет адаптивным оптимизаторам лучше оценить значения `learning rate` для разных параметров

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/neural_network_warmup.png" width="1000">

<center><p><em>Source: <a href="https://arxiv.org/pdf/1706.02677.pdf">Accurate, Large Minibatch SGD: Training ImageNet in 1 Hour</a></p> </em></center>


$kn$ на картинке &mdash; это размер одного батча.

### Взаимодействие learning schedule и адаптивного изменения learning rate

И то, и другое меняет `learning rate`:
 
`learning scheduler` &mdash; глобально,

 адаптивные оптимизаторы &mdash; для каждого веса отдельно.

Часто их применяют вместе, особенно в случае критерия ранней остановки и WarmUp

Однако в случае циклического режима обучения так делают не всегда. Дело в том, что одна из его задач &mdash; как раз избежать использования адаптивных оптимизаторов, требующих бОльше памяти и дополнительных вычислений. 

Однако никаких препятствий к использованию того же Adam в компании вместе с циклическим режимом обучения нет. В [исходной статье](https://arxiv.org/pdf/2004.02401.pdf) так делают. 

<font size = "6">Ссылки:</font>

[A journey into Optimization algorithms for Deep Neural Networks](https://theaisummer.com/optimization/)

[Optimizers Explained — Adam, Momentum and Stochastic Gradient Descent](https://medium.com/geekculture/a-2021-guide-to-improving-cnns-optimizers-adam-vs-sgd-495848ac6008)

[Батч-нормализация. In-layer normalization techniques for training very deep neural networks](https://theaisummer.com/normalization/)

[Циклический learning rate](https://towardsdatascience.com/adaptive-and-cyclical-learning-rates-using-pytorch-2bf904d18dee)

[Разные функции активации, затухающие и взрывающиеся градиенты и т.д.](https://www.kdnuggets.com/2022/06/activation-functions-work-deep-learning.html)

[Визуализация разных оптимизаторов в ipynb, но на tensorflow](https://nbviewer.jupyter.org/github/ilguyi/optimizers.numpy/blob/master/optimizer.tf.all.opt.plot.ipynb)